# Pilot subject 06 example script

## Data import and preprocessing

In [ ]:
import mne
from mne.preprocessing import ICA

import os

import numpy as np

os.chdir('F:\\TBS_KB\\')

MNE provides a broad range of montage files, supporting the 10-05, 10-20 and EGI's geodesic elctrode positioning system as well as many different manufacturers' caps (EGI, biosemi, easycap).

For this example, I want to show that the MNE info structure can be easily adapted with custom montages.

In [ ]:
fname = 'TMSKB0006.vhdr'
kind = 'standard_1020'
eog = ['VEOG', 'LHZ', 'RHZ']

When reading event codes they are stripped to their integer parts (S1 -> 1).

In [ ]:
montage = mne.channels.read_montage(kind=kind)
raw = mne.io.read_raw_brainvision(fname, preload=True, montage=montage, eog=eog)

Let's look at the raw data.

In [ ]:
raw.plot(duration=20, n_channels=24, scalings=dict(eeg=100e-6));

Before starting to preprocess the data, pick the channels to be filtered and refreferenced. Apply a bandpass filter (0.1, 30Hz) and rereference to the mean of the two mastoid positions.

In [ ]:
picks = mne.pick_types(raw.info, eeg=True)

raw.filter(1, 20) 
raw.set_eeg_reference(ref_channels=['M1','M2'], projection=False)

In [ ]:
raw.copy().plot_sensors(kind='3d', ch_groups='position', show_names=True);
raw.copy().plot_sensors(kind='topomap', ch_groups='position', show_names=True);

Next, put together all arguments necessary for starting an Independent Component Analysis (ICA). For the purpose of this example, let's keep the parameters for reducing the amount of data samples (decim) and for detecting data segments which should be excluded (reject) at None.

In [ ]:
n_components = 10  
method = 'extended-infomax'
decim = None
reject = None

ica = ICA(n_components=n_components, method=method)

In [ ]:
ica.fit(raw, decim=decim, reject=reject)

In [ ]:
ica.plot_components(outlines="skirt");

For selecting artifact components we check the topographies, frequency spectra and temporal distribution of components. That takes a little bit of practice, but most artifact components are quite distinct and easy to spot.

In [ ]:
ica.plot_properties(raw, [5,7,9])

But sometimes you want a more objective way of rating artifact components. So let's look at what automatic algorithms (based on channel activity) classify as artifacts.

In [ ]:
from scipy.stats import pearsonr

corr = lambda x, y: np.array([pearsonr(a, y.ravel()) for a in x])[:, 0]

# The find_sources method returns and array of correlation
# scores for each ICA source.
eog_scores = ica.score_sources(raw, target='VEOG', score_func=corr)

# Get the component index of the maximum correlation with the ECG
eog_source_idx = np.abs(eog_scores).argmax()

# Mark the sources with the strongest correlations
ica.plot_sources(raw, exclude=[eog_source_idx], picks=[0,1,2,3,4,5,6,7,8,9], start=100, stop=200);

In [ ]:
# Blinks
bad_idxV, scoresV = ica.find_bads_eog(raw, 'VEOG', threshold=2)
# Saccades
bad_idxH, scoresH = ica.find_bads_eog(raw, 'LHZ', threshold=2)

In [ ]:
ica.plot_scores(scoresV, title='ICA component scores, vertical (blinks)',
                exclude=bad_idxV, # Mark the identified eog artifact components red
                show=True, 
                axhline=[0.2, -0.2]); # Plot a horizontal line to mark a score threshold
ica.plot_scores(scoresH, title='ICA component scores, horizontal (saccades)',
                exclude=bad_idxH,
                show=True, 
                axhline=[0.2, -0.2]);

Here are some safe picks, which were consistently marked as possible artifacts by algorithms and by conventional visual inspection.

In [ ]:
ica.exclude = [0,2,3,5,7,9]

Let's compare raw and corrected data ...

In [ ]:
raw.plot(start=1000, duration=10);

In [ ]:
ica.apply(raw, exclude=ica.exclude).plot(start=1000, duration=10);

## Epoching

Moving on to finding the EEG segments we want to analyze. Here, we first have to import the important information from the event file.

In [ ]:
event_id = {'reward_right': 7, 'reward_left': 9, 'no_reward_right': 6, 'no_reward_left': 8}
events = mne.find_events(raw, shortest_event=0, initial_event=True)

events is simply an array (time in samples, zero, trigger);

In [ ]:
events[:10]

For a sanity check, let's look at the distribution of the first 100 events over time.

In [ ]:
mne.viz.plot_events(events[:1000], sfreq=raw.info['sfreq'], event_id=event_id, show=True);

In [ ]:
tmin, tmax = -0.2, 0.8
baseline = -0.2, -0.1
epochs_evoked = mne.Epochs(raw, events=events, event_id=event_id, picks=picks,
                           tmin=tmin, tmax=tmax, baseline=baseline)
epochs_tfr = mne.Epochs(raw, events=events, event_id=event_id, picks=picks,
                        tmin=-0.5, tmax=1.5, baseline=None)

Now we can plot some data.

In [ ]:
epochs_evoked['reward_right', 'reward_left'].plot_psd(fmin=0, fmax=60);
epochs_evoked['no_reward_right', 'no_reward_left'].plot_psd(fmin=0, fmax=60);

In [ ]:
epochs_evoked['reward_right', 'reward_left'].plot_image(picks=[7]);
epochs_evoked['no_reward_right', 'no_reward_left'].plot_image(picks=[7]);

In [ ]:
reward = epochs_evoked['reward_left','reward_right'].average()
noreward = epochs_evoked['no_reward_left','no_reward_right'].average()

difference_feedback = mne.combine_evoked((reward,noreward),[1,-1])

In [ ]:
colors = dict(Reward = 'darkblue', No_reward = 'darkred', 
              Difference_Wave = 'black')
maze_dict = {'Reward': reward, 'No_reward': noreward, 
               'Difference_Wave': difference_feedback}

linestyles = dict(Reward = '-', No_reward = '--', 
                  Difference_Wave = '-')

ylim = dict(eeg=[-7,7])

mne.viz.plot_compare_evokeds(maze_dict, picks=[7], 
                             truncate_yaxis=False, truncate_xaxis=False,
                             colors=colors, linestyles=linestyles,
                             invert_y=False, ylim = ylim,
                             title='Feedback related responses',
                             show_sensors=True);

Perform a rough division of blocks based on trial counts.